## Description: prepare the notebook for manual annotation

In this notebook, three elements are added to the previously generated dataset:
- check that the chapter and verse_num columns are integers
- sign_info column, based on a recent version of the function
- reconstructed_verse column which is a gcons version of the verse with brackets indicating the reconstructed sections
- the spatial_arg_type column for annotating goal, source, location, etc. 
- remove the rows which have "MN" in preposition_1 and no preposition_2

In [23]:
import pandas as pd

from itertools import chain

from tfob import TFOb,  get_dss

DSS = get_dss()

PATH = 'data/biblical_datasets/2_chronicles/annotation_df_history/2_Chronicles_dataset.csv' # Modify before executing the notebook

**Locating corpus resources ...**

Name,# of nodes,# slots / node,% coverage
scroll,1001,1428.81,100
lex,10450,129.14,94
fragment,11182,127.91,100
line,52895,27.04,100
clause,125,12.85,0
cluster,101099,6.68,47
phrase,315,5.10,0
word,500995,2.81,99
sign,1430241,1.00,100


In [24]:
#df_1 = pd.read_csv('../2_datasets/generate_datasets/data/biblical_datasets/Jeremiah_dataset.csv').fillna("")
df = pd.read_csv(PATH).fillna("")

In [25]:
# Make the values in "chapter" and "verse_num" integers
df['chapter'] = df['chapter'].astype(int)
df['verse_num'] = df['verse_num'].astype(int)

In [26]:
# Reorganise the dataset by chapter, verse_num and scroll

# Custom sorting logic for "scroll" column
df['scroll_order'] = df['scroll'].apply(lambda x: 0 if x == "MT" else 1)

# Sort by "scroll_order", "chapter", and "verse_num"
df = df.sort_values(by=['chapter', 'verse_num', 'scroll_order'])

# Drop the temporary 'scroll_order' column
df = df.drop(columns='scroll_order')

df = df.reset_index(drop=True)

In [27]:
pd.set_option("display.max_columns", None)

In [28]:
df.head(20)

,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,gcons_clause,subject,complement,cmpl_lex,dir_he,cmpl_constr,cmpl_nt,cmpl_anim,cmpl_det,cmpl_indiv,cmpl_complex,dir_he_dss,sign_info,stem,tense,preposition_1,preposition_2,preposition_3,preposition_4,preposition_5,preposition_6,preposition_7,cmpl_translation,motion_type,comments
0,406870,HLK[,MT,2_Chronicles,1,3,JLKW,W JLKW CLMH W KL H QHL <MW L BMH >CR B GB<WN K...,W JLKW CLMH W KL H QHL <MW L BMH,CLMH W KL H QHL <MW,L BMH,L H BMH/,0.0,prep,,,det,subs,simple,,,qal,wayq,L,,,,,,,,,
1,406902,<LH[,MT,2_Chronicles,1,4,H<LH,>BL >RWN H >LHJM H<LH DWJD M QRJT_J<RJM B HKJN...,>BL >RWN H >LHJM H<LH DWJD M QRJT_J<RJM,DWJD,M QRJT_J<RJM,MN QRJT_J<RJM/,0.0,prep,topo,inanim,det,nmpr,simple,,,hif,perf,MN,,,,,,,,,
2,406940,<LH[,MT,2_Chronicles,1,6,J<L,W J<L CLMH CM <L MZBX H NXCT L PNJ JHWH >CR L ...,W J<L CLMH CM <L MZBX H NXCT L PNJ JHWH,CLMH,<L MZBX H NXCT,<L MZBX/ H NXCT/,0.0,prep,,,det,subs,complex,,,hif,wayq,<L,,,,,,,,,
3,406955,<LH[,MT,2_Chronicles,1,6,J<L,W J<L CLMH CM <L MZBX H NXCT L PNJ JHWH >CR L ...,W J<L <LJW <LWT >LP,,<LJW,<L,0.0,prep,,,det,,simple,,,hif,wayq,<L,,,,,,,,,
4,407015,JY>[,MT,2_Chronicles,1,10,>Y>H,<TH XKMH W MD< TN LJ W >Y>H L PNJ H <M H ZH W ...,W >Y>H L PNJ H <M H ZH,,L PNJ H <M H ZH,L PNH/ H <M/ H ZH,0.0,prep,,,det,subs,complex,,,qal,impf,L,,,,,,,,,
5,407023,BW>[,MT,2_Chronicles,1,10,>BW>H,<TH XKMH W MD< TN LJ W >Y>H L PNJ H <M H ZH W ...,W >BW>H,,no complement,no complement,,,,,,,,,,qal,impf,,,,,,,,,,
6,407105,BW>[,MT,2_Chronicles,1,13,JB>,W JB> CLMH L BMH >CR B GB<WN JRWCLM M L PNJ >H...,W JB> CLMH L BMH JRWCLM,CLMH,JRWCLM,JRWCLM/,0.0,vc,topo,inanim,det,nmpr,simple,,,qal,wayq,,,,,,,,,,
7,407143,NWX[,MT,2_Chronicles,1,14,JNJXM,W J>SP CLMH RKB W PRCJM W JHJ LW >LP W >RB< M>...,W JNJXM B <RJ H RKB W <M H MLK B JRWCLM,,B <RJ H RKB W <M H MLK B JRWCLM,B <JR/ H RKB/ W <M H MLK/ B JRWCLM/,0.0,prep + prep + prep,topo,inanim,det,subs,complex,,,hif,wayq,B,<M,B,,,,,,,
8,407205,<LH[,MT,2_Chronicles,1,17,J<LW,W J<LW W JWYJ>W M MYRJM MRKBH B CC M>WT KSP W ...,W J<LW,,no complement,no complement,,,,,,,,,,hif,wayq,,,,,,,,,,
9,407207,JY>[,MT,2_Chronicles,1,17,JWYJ>W,W J<LW W JWYJ>W M MYRJM MRKBH B CC M>WT KSP W ...,W JWYJ>W M MYRJM MRKBH B CC M>WT KSP,,M MYRJM,MN MYRJM/,0.0,prep,topo,inanim,det,nmpr,simple,,,hif,wayq,MN,,,,,,,,,


In [29]:
df.columns

Index(['verb_id', 'lex', 'scroll', 'book', 'chapter', 'verse_num',
       'gcons_verb', 'gcons_verse', 'gcons_clause', 'subject', 'complement',
       'cmpl_lex', 'dir_he', 'cmpl_constr', 'cmpl_nt', 'cmpl_anim', 'cmpl_det',
       'cmpl_indiv', 'cmpl_complex', 'dir_he_dss', 'sign_info', 'stem',
       'tense', 'preposition_1', 'preposition_2', 'preposition_3',
       'preposition_4', 'preposition_5', 'preposition_6', 'preposition_7',
       'cmpl_translation', 'motion_type', 'comments'],
      dtype='object')

In [30]:
# This is the most recent sign_info function which uses the .rec (1 October 2024)

def sign_info(verb):
    sign_rec_words = []
    
    scroll = verb.to_scrolls.scroll[0]
    verse = TFOb.section([verb.book[0], verb.chapter[0], verb.verse[0]], DSS, scroll)
    
    for word in verse.to_words:
        signs = word.to_signs
        sign_rec = []
        for sign in signs:
            if sign.type[0] == "cons": # only keeps consonants
                if sign.rec[0]:
                    sign_rec.append("1")
                else:
                    sign_rec.append("0")
        if sign_rec:
            sign_rec_words.append("".join(sign_rec))
        
    return " ".join(sign_rec_words).strip()

In [31]:
# Updates the dataframe (change the old sign_info column to one containing the return of the new function)

def update_sign_info(row):
    if row['scroll'] != 'MT':
        return sign_info(TFOb(int(row['verb_id']), DSS))
    return row['sign_info']

# Apply the function to the dataframe
df['sign_info'] = df.apply(update_sign_info, axis=1)

In [32]:
# Initialize an empty list to collect mismatch information
mismatches = []

# Function to check the lengths of sign_info and gcons_verse
def check_length(row):
    if row['scroll'] != 'MT':
        if len(row['sign_info']) != len(row['gcons_verse']):
            mismatches.append(f"Mismatch found in row {row.name}: sign_info='{row['sign_info']}' (len={len(row['sign_info'])}), gcons_verse='{row['gcons_verse']}' (len={len(row['gcons_verse'])})")

# Apply the function to the dataframe
df.apply(check_length, axis=1)

# Check if there are any mismatches and print them
if mismatches:
    print("Mismatches found:")
    for mismatch in mismatches:
        print(mismatch)
else:
    print("No mismatches found.")

No mismatches found.


In [33]:
# The reconstruct_verse function adds a gcons version of the verse with brackets indicating the reconstructed sections

# Add a column with the verse plus brackets indicating reconstructed letters

def reconstruct_verse(sign_info, verse):
    rec_verse = []
    is_reconstructed = False

    for c_verse, c_sign in zip(verse, sign_info):
        if c_sign == "1" and not is_reconstructed:
            is_reconstructed = True
            rec_verse.append("[")
        if c_sign == "0" and is_reconstructed:
            is_reconstructed = False
            if rec_verse[-1] == " ":
                rec_verse.pop()
                rec_verse.extend(["]", " "])
            else:
                rec_verse.append("]")
        rec_verse.append(c_verse)

    if is_reconstructed:
        rec_verse.append("]")
    return "".join(rec_verse)


df["reconstructed_verse"] = ""

df["reconstructed_verse"] = df.apply(lambda row: reconstruct_verse(row.sign_info, row.gcons_verse) if row.book != "MT" else "", axis=1)

In [34]:
# Delete the DSS duplicated

verb_ids = df.verb_id.to_list()

duplicates = set([verb_id for verb_id in verb_ids if verb_ids.count(verb_id) > 1])


df = df[~((df.gcons_clause == "None") & (df.subject == "None") & (df.verb_id.isin(duplicates)))] # delete duplicates
df.reset_index(drop=True, inplace=True) # reset the index for AnnotEasy

In [35]:
df.columns

Index(['verb_id', 'lex', 'scroll', 'book', 'chapter', 'verse_num',
       'gcons_verb', 'gcons_verse', 'gcons_clause', 'subject', 'complement',
       'cmpl_lex', 'dir_he', 'cmpl_constr', 'cmpl_nt', 'cmpl_anim', 'cmpl_det',
       'cmpl_indiv', 'cmpl_complex', 'dir_he_dss', 'sign_info', 'stem',
       'tense', 'preposition_1', 'preposition_2', 'preposition_3',
       'preposition_4', 'preposition_5', 'preposition_6', 'preposition_7',
       'cmpl_translation', 'motion_type', 'comments', 'reconstructed_verse'],
      dtype='object')

In [36]:
# Add the spatial_arg_type column

df["spatial_arg_type"] = ""

In [37]:
df.columns

Index(['verb_id', 'lex', 'scroll', 'book', 'chapter', 'verse_num',
       'gcons_verb', 'gcons_verse', 'gcons_clause', 'subject', 'complement',
       'cmpl_lex', 'dir_he', 'cmpl_constr', 'cmpl_nt', 'cmpl_anim', 'cmpl_det',
       'cmpl_indiv', 'cmpl_complex', 'dir_he_dss', 'sign_info', 'stem',
       'tense', 'preposition_1', 'preposition_2', 'preposition_3',
       'preposition_4', 'preposition_5', 'preposition_6', 'preposition_7',
       'cmpl_translation', 'motion_type', 'comments', 'reconstructed_verse',
       'spatial_arg_type'],
      dtype='object')

In [38]:
# Remove the columns that have MN as preposition 1 and not other prepositions

In [39]:
# have a look at the data
df_test = df[(df.preposition_1 == "MN") & (df.preposition_2 == "MN")]
df_test.head(5)
#len(df_test)

,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,gcons_clause,subject,complement,cmpl_lex,dir_he,cmpl_constr,cmpl_nt,cmpl_anim,cmpl_det,cmpl_indiv,cmpl_complex,dir_he_dss,sign_info,stem,tense,preposition_1,preposition_2,preposition_3,preposition_4,preposition_5,preposition_6,preposition_7,cmpl_translation,motion_type,comments,reconstructed_verse,spatial_arg_type
159,413959,<BR[,MT,2_Chronicles,15,8,J<BR,W K CM< >S> H DBRJM H >LH W H NBW>H <DD H NBJ>...,W J<BR H CQWYJM M KL >RY JHWDH W BNJMN W MN H ...,,M KL >RY JHWDH W BNJMN W MN H <RJM,MN KL/ >RY/ JHWDH/ W BNJMN/ W MN H <JR/,0.0,prep + prep,"pers,gens,topo pers,gens,topo",,det,subs,complex,,,hif,wayq,MN,MN,,,,,,,,,,
309,418340,BW>[,MT,2_Chronicles,24,6,HBJ>,W JQR> H MLK L JHWJD< H R>C W J>MR LW MDW< L> ...,L HBJ> M JHWDH W M JRWCLM >T MF>T MCH <BD JHWH...,,M JHWDH W M JRWCLM,MN JHWDH/ W MN JRWCLM/,0.0,prep + prep,"pers,gens,topo topo",inanim,det,nmpr,complex,,,hif,infc,MN,MN,,,,,,,,,,


In [40]:
len(df_test)

2

In [41]:
# Identify the rows to drop based on the condition
index_to_drop = df[(df.preposition_1 == "MN") & (df.preposition_2 == "")].index
index_to_drop_2 = df[(df.preposition_1 == "MN") & (df.preposition_2 == "MN") & (df.preposition_3 == "")].index
index_to_drop_3 = df[(df.preposition_1 == "MN") & (df.preposition_2 == "MN") & (df.preposition_3 == "MN") & (df.preposition_4 == "")].index

# Drop the rows by index
df = df.drop(index_to_drop)
df = df.drop(index_to_drop_2)
df = df.drop(index_to_drop_3)

# Reset the index
df = df.reset_index(drop=True)

In [42]:
# Verification
df[(df.preposition_1 == "MN") & (df.preposition_2 == "")]

,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,gcons_clause,subject,complement,cmpl_lex,dir_he,cmpl_constr,cmpl_nt,cmpl_anim,cmpl_det,cmpl_indiv,cmpl_complex,dir_he_dss,sign_info,stem,tense,preposition_1,preposition_2,preposition_3,preposition_4,preposition_5,preposition_6,preposition_7,cmpl_translation,motion_type,comments,reconstructed_verse,spatial_arg_type


In [43]:
df.head(20)

,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,gcons_clause,subject,complement,cmpl_lex,dir_he,cmpl_constr,cmpl_nt,cmpl_anim,cmpl_det,cmpl_indiv,cmpl_complex,dir_he_dss,sign_info,stem,tense,preposition_1,preposition_2,preposition_3,preposition_4,preposition_5,preposition_6,preposition_7,cmpl_translation,motion_type,comments,reconstructed_verse,spatial_arg_type
0,406870,HLK[,MT,2_Chronicles,1,3,JLKW,W JLKW CLMH W KL H QHL <MW L BMH >CR B GB<WN K...,W JLKW CLMH W KL H QHL <MW L BMH,CLMH W KL H QHL <MW,L BMH,L H BMH/,0.0,prep,,,det,subs,simple,,,qal,wayq,L,,,,,,,,,,,
1,406940,<LH[,MT,2_Chronicles,1,6,J<L,W J<L CLMH CM <L MZBX H NXCT L PNJ JHWH >CR L ...,W J<L CLMH CM <L MZBX H NXCT L PNJ JHWH,CLMH,<L MZBX H NXCT,<L MZBX/ H NXCT/,0.0,prep,,,det,subs,complex,,,hif,wayq,<L,,,,,,,,,,,
2,406955,<LH[,MT,2_Chronicles,1,6,J<L,W J<L CLMH CM <L MZBX H NXCT L PNJ JHWH >CR L ...,W J<L <LJW <LWT >LP,,<LJW,<L,0.0,prep,,,det,,simple,,,hif,wayq,<L,,,,,,,,,,,
3,407015,JY>[,MT,2_Chronicles,1,10,>Y>H,<TH XKMH W MD< TN LJ W >Y>H L PNJ H <M H ZH W ...,W >Y>H L PNJ H <M H ZH,,L PNJ H <M H ZH,L PNH/ H <M/ H ZH,0.0,prep,,,det,subs,complex,,,qal,impf,L,,,,,,,,,,,
4,407023,BW>[,MT,2_Chronicles,1,10,>BW>H,<TH XKMH W MD< TN LJ W >Y>H L PNJ H <M H ZH W ...,W >BW>H,,no complement,no complement,,,,,,,,,,qal,impf,,,,,,,,,,,,
5,407105,BW>[,MT,2_Chronicles,1,13,JB>,W JB> CLMH L BMH >CR B GB<WN JRWCLM M L PNJ >H...,W JB> CLMH L BMH JRWCLM,CLMH,JRWCLM,JRWCLM/,0.0,vc,topo,inanim,det,nmpr,simple,,,qal,wayq,,,,,,,,,,,,
6,407143,NWX[,MT,2_Chronicles,1,14,JNJXM,W J>SP CLMH RKB W PRCJM W JHJ LW >LP W >RB< M>...,W JNJXM B <RJ H RKB W <M H MLK B JRWCLM,,B <RJ H RKB W <M H MLK B JRWCLM,B <JR/ H RKB/ W <M H MLK/ B JRWCLM/,0.0,prep + prep + prep,topo,inanim,det,subs,complex,,,hif,wayq,B,<M,B,,,,,,,,,
7,407205,<LH[,MT,2_Chronicles,1,17,J<LW,W J<LW W JWYJ>W M MYRJM MRKBH B CC M>WT KSP W ...,W J<LW,,no complement,no complement,,,,,,,,,,hif,wayq,,,,,,,,,,,,
8,407233,JY>[,MT,2_Chronicles,1,17,JWYJ>W,W J<LW W JWYJ>W M MYRJM MRKBH B CC M>WT KSP W ...,B JDM JWYJ>W,,no complement,no complement,,,,,,,,,,hif,impf,,,,,,,,,,,,
9,407671,BW>[,MT,2_Chronicles,2,15,NBJ>M,W >NXNW NKRT <YJM MN H LBNWN K KL YRKK W NBJ>M...,W NBJ>M LK RPSDWT <L JM JPW,,LK,L,0.0,prep,,,det,,simple,,,hif,impf,L,,,,,,,,,,,


In [44]:
df.to_csv(PATH, index=False)